In [ ]:
#@title 蒼狼 AI V2.2 可觀測性分析平台 - 一鍵啟動 (修正版)
import os
import shutil
import time
import requests
import subprocess
import pytz
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from google.colab import output

# --- 全域設定 ---
GIT_REPO_URL = "https://github.com/hsp1234-web/wolfAI_v1.git"
PROJECT_PARENT_DIR = "/content/wolf_project"
PROJECT_DIR_NAME = "wolfAI_v1"
FULL_PROJECT_PATH = os.path.join(PROJECT_PARENT_DIR, PROJECT_DIR_NAME)
TAIPEI_TZ = pytz.timezone('Asia/Taipei')

# --- UI 元件定義 ---
style = {'description_width': 'initial'}

# 新增：操作模式選擇
operation_mode_selection = widgets.Dropdown(
    options=[
        ('暫存模式 (資料不保存)', 'transient'),
        ('持久模式 (需授權 Google Drive)', 'persistent')
    ],
    value='transient',
    description='1. 選擇操作模式:',
    style=style
)

# 新增：分支選擇
branch_selection = widgets.Text(
    value='main',
    description='2. GitHub 分支:',
    style=style
)

# 啟動模式選擇
launch_mode_selection = widgets.RadioButtons(
    options=['一般模式 (normal)', '除錯模式 (debug)'],
    description='3. 選擇啟動模式:',
    disabled=False,
    style=style
)

confirm_button = widgets.Button(
    description="確認並開始啟動",
    disabled=False,
    button_style='success',
    tooltip='點擊此按鈕開始安裝與部署',
    icon='rocket'
)

info_output = widgets.Output()

# --- Helper 函數 ---
def get_taipei_time_str():
    """返回格式化的台北時間字串"""
    return datetime.now(TAIPEI_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')

def run_command(command, cwd=None, text=True):
    """執行 shell 命令並即時串流輸出"""
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        shell=True,
        text=text,
        bufsize=1,
        universal_newlines=True,
        cwd=cwd
    )
    for line in iter(process.stdout.readline, ''):
        print(line, end='')
    process.stdout.close()
    return process.wait()

def start_deployment(b):
    """點擊確認按鈕後執行的主程序"""
    with info_output:
        clear_output(wait=True)
        # 禁用按鈕以防重複點擊
        confirm_button.disabled = True
        operation_mode_selection.disabled = True
        branch_selection.disabled = True
        launch_mode_selection.disabled = True

        operation_mode = operation_mode_selection.value
        branch_name = branch_selection.value
        launch_mode = launch_mode_selection.value.split('(')[1].replace(')', '')

        print(f"[{get_taipei_time_str()}] ✅ 設定已確認")
        print(f"   - 操作模式: {operation_mode}")
        print(f"   - 啟動模式: {launch_mode}")
        print(f"   - 程式碼分支: {branch_name}")
        print("-" * 50)

        # 將選擇的操作模式設定為環境變數，供後端應用讀取
        os.environ['OPERATION_MODE'] = operation_mode

        print(f"\n[{get_taipei_time_str()}] STEP 1: 檢查 Google Drive (操作模式: {operation_mode})...")
        if operation_mode == "persistent":
            try:
                from google.colab import drive
                drive.mount('/content/drive', force_remount=True)
                print(f"[{get_taipei_time_str()}] ✅ SUCCESS: Google Drive 已成功掛載。")
                print(f"[{get_taipei_time_str()}] INFO: 請確保您已在 Colab Secrets 中設定持久模式所需的金鑰 (例如 GOOGLE_SERVICE_ACCOUNT_JSON_CONTENT, WOLF_IN_FOLDER_ID 等)。")
            except Exception as e:
                print(f"[{get_taipei_time_str()}] ❌ ERROR: 掛載 Google Drive 失敗: {e}")
                # 重新啟用按鈕
                confirm_button.disabled = False
                operation_mode_selection.disabled = False
                branch_selection.disabled = False
                launch_mode_selection.disabled = False
                return
        else:
            print(f"[{get_taipei_time_str()}] INFO: 暫存模式，跳過 Google Drive 掛載。")
            if not os.getenv('COLAB_GOOGLE_API_KEY'):
                print(f"[{get_taipei_time_str()}] ⚠️ WARNING: 未偵測到 COLAB_GOOGLE_API_KEY，AI 分析功能將受限。")

        print(f"\n[{get_taipei_time_str()}] STEP 2: 複製專案程式碼 (分支: {branch_name})...")
        if os.path.exists(FULL_PROJECT_PATH):
            shutil.rmtree(FULL_PROJECT_PATH)
        clone_command = f"git clone --depth 1 -b {branch_name} {GIT_REPO_URL} {FULL_PROJECT_PATH}"
        if run_command(clone_command) != 0:
            print(f"[{get_taipei_time_str()}] ❌ FATAL: 專案複製失敗。")
            confirm_button.disabled = False
            operation_mode_selection.disabled = False
            branch_selection.disabled = False
            launch_mode_selection.disabled = False
            return
        print(f"[{get_taipei_time_str()}] ✅ SUCCESS: 專案已成功複製。")

        print(f"\n[{get_taipei_time_str()}] STEP 3: 執行啟動腳本 (scripts/start.sh)...")
        start_script_path = os.path.join(FULL_PROJECT_PATH, 'scripts', 'start.sh')
        os.chmod(start_script_path, 0o755)
        start_command = f"{start_script_path} --mode={launch_mode}"
        if run_command(start_command, cwd=FULL_PROJECT_PATH) != 0:
            print(f"[{get_taipei_time_str()}] ❌ FATAL: 啟動腳本執行失敗，請檢查日誌。")
            confirm_button.disabled = False
            operation_mode_selection.disabled = False
            branch_selection.disabled = False
            launch_mode_selection.disabled = False
            return
        print(f"[{get_taipei_time_str()}] ✅ SUCCESS: 啟動腳本已執行。")

        print(f"\n[{get_taipei_time_str()}] STEP 4: 健康度偵測 (Health Check)...")
        time.sleep(15) # 等待服務啟動
        backend_health_url = "http://localhost:8000/api/health"
        max_wait_seconds = 120
        start_time = time.time()
        backend_ready = False
        while time.time() - start_time < max_wait_seconds:
            try:
                response = requests.get(backend_health_url, timeout=5)
                if response.status_code == 200:
                    print(f"[{get_taipei_time_str()}] ✅ INFO: 後端服務健康檢查通過！")
                    backend_ready = True
                    break
                else:
                    print(f"[{get_taipei_time_str()}] ⏳ WARNING: 後端健康檢查異常，狀態碼: {response.status_code}。重試中...")
            except requests.exceptions.RequestException:
                print(f"[{get_taipei_time_str()}] ⏳ INFO: 後端服務尚未就緒，重試中...")
            time.sleep(10)

        print(f"\n[{get_taipei_time_str()}] STEP 5: 顯示結果...")
        border = "="*50
        if backend_ready:
            print(f"\n{border}")
            display(HTML("<h2>✅ 應用程式後端已就緒！</h2>"))
            try:
                # 使用 google.colab.output.eval_js 來獲取代理 URL
                frontend_url = output.eval_js(f'google.colab.kernel.proxyPort(3000, {{ "cache_bust": True }})')
                display(HTML(f"<p>🔗 <b>前端訪問網址:</b> <a href='{frontend_url}' target='_blank'>{frontend_url}</a></p>"))
                print("注意：如果前端頁面未立即載入，請稍等片刻或嘗試刷新頁面。")
            except Exception as e:
                display(HTML(f"<p>⚠️ 無法自動獲取前端URL，請手動在Colab輸出上方尋找。錯誤: {e}</p>"))
            print(f"{border}\n")
        else:
            print(f"\n{border}")
            display(HTML("<h2>❌ 後端服務未能成功啟動。</h2>"))
            print("請檢查日誌以了解錯誤。可嘗試使用'除錯模式'獲取更多資訊。")
            print(f"{border}\n")

        # 重新啟用按鈕
        confirm_button.disabled = False
        operation_mode_selection.disabled = False
        branch_selection.disabled = False
        launch_mode_selection.disabled = False

# --- 主執行區 ---
confirm_button.on_click(start_deployment)
display(
    widgets.VBox([
        operation_mode_selection,
        branch_selection,
        launch_mode_selection,
        confirm_button
    ]),
    info_output
)
